In [1]:
import sys
sys.path.append('../')

In [2]:
from textpair.single.paddle_bow import PaddleBowSim
from textpair.single.ann import Ann
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
VOCAB_FILE = '../../../model_files/paddle_models/sim_net/data/term2id.dict'
MODEL_PATH = '../../../model_files/paddle_models/sim_net/model_files/simnet_bow_pairwise_pretrained_model/'

In [4]:
sim = PaddleBowSim(VOCAB_FILE, MODEL_PATH)

In [5]:
text1 = "人民团体是什么"
text2 = "人民团体是指"
ann1 = Ann(text1)
ann2 = Ann(text2)

In [6]:
res1 = sim(ann1, ann2)
score1 = res1["score"]
vec1 = res1["left_vec"]

Building prefix dict from the default dictionary ...
2019-06-25 23:13:25,728-DEBUG: Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LINAN\AppData\Local\Temp\jieba.cache
2019-06-25 23:13:25,735-DEBUG: Loading model from cache C:\Users\LINAN\AppData\Local\Temp\jieba.cache
Loading model cost 0.637 seconds.
2019-06-25 23:13:26,370-DEBUG: Loading model cost 0.637 seconds.
Prefix dict has been built succesfully.
2019-06-25 23:13:26,371-DEBUG: Prefix dict has been built succesfully.


In [7]:
score1

0.9515146613121033

In [8]:
for i in tqdm(range(3000)):
    ann1 = Ann(text1)
    ann2 = Ann(text2)
    res = sim(ann1, ann2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:02<00:00, 1023.21it/s]


In [9]:
vec1

array([[-1.70560688e-01, -3.74650955e-02,  5.14803708e-01,
        -8.30972493e-02, -6.99904338e-02,  2.11101443e-01,
         2.78322875e-01,  1.80265248e-01,  3.15677464e-01,
        -2.06045359e-02,  6.17800415e-01,  3.23109329e-04,
         4.44537997e-02,  3.98692936e-01,  1.59228854e-02,
         4.28976566e-01,  3.96663457e-01, -7.29419813e-02,
        -9.72113088e-02, -1.44232765e-01,  5.54060973e-02,
         4.19091970e-01, -2.40897954e-01,  2.23761201e-01,
         2.55655289e-01,  4.06692773e-02,  5.28913081e-01,
         8.28170717e-01,  2.57944673e-01,  5.23537695e-01,
         3.37901056e-01,  2.48735070e-01, -5.66174805e-01,
         7.18245506e-02, -6.48569465e-01,  7.92640209e-01,
         5.29523909e-01, -4.94030625e-01,  3.55076194e-02,
        -2.81550497e-01, -2.68068969e-01,  9.48422253e-02,
         4.47783470e-01, -4.00969684e-02,  4.84471142e-01,
        -2.65492886e-01,  3.08129102e-01,  2.60512531e-02,
         1.25101715e-01, -1.20797649e-01, -5.25811352e-0

In [10]:
ann1 = Ann(text2)
ann2 = Ann(text1)
res2 = sim(ann1, ann2)
score2 = res2["score"]
vec2 = res2["left_vec"]

In [11]:
score2

0.9515146613121033

In [12]:
vec2

array([[-0.07193187, -0.02850994,  0.41007343, -0.23986797, -0.18175876,
         0.3844999 ,  0.53248185,  0.16439942,  0.08232097, -0.10547136,
         0.80433553, -0.07167187,  0.12474287,  0.13358435,  0.15730448,
         0.48062217,  0.31346595, -0.12053117,  0.06695841, -0.1720372 ,
         0.31366536,  0.45438138, -0.05947053,  0.34327656,  0.32160544,
        -0.032986  ,  0.23279564,  0.9614878 ,  0.1846422 ,  0.46296197,
         0.40326563,  0.14319545, -0.77499175,  0.4210466 , -0.50555253,
         0.6712532 ,  0.43920186, -0.2757621 ,  0.15027244, -0.18368083,
        -0.19620834, -0.03260714,  0.45094398,  0.08652598,  0.6285967 ,
        -0.31752992,  0.2115895 , -0.09861028,  0.28990874, -0.10580373,
        -0.1831078 , -0.14328665, -0.6035165 , -0.13392395,  0.09366333,
        -0.43277267,  0.111536  ,  0.31390622,  0.01611441, -0.6012186 ,
        -0.24603441, -0.23231858, -0.5872389 ,  0.18065363, -0.30484894,
         0.00930201, -0.28493938,  0.3666643 , -0.1

In [13]:
score_vec1_vec2 = (cosine_similarity(vec1, vec2)[0, 0] + 1) / 2

In [14]:
score1, score2, score_vec1_vec2

(0.9515146613121033, 0.9515146613121033, 0.9515146613121033)